## How to create a data point for training cyclesNN
In this notebook, we show how to specify a desired N fertilization application, run cycles with it, and read the final grain yield.

In [ ]:
from pathlib import Path
import subprocess
import shutil
import stat

from cyclesgym.managers import *
from cyclesgym.paths import CYCLES_PATH

INPUT_DIR = CYCLES_PATH.joinpath('input')
OUTPUT_DIR = CYCLES_PATH.joinpath('output')

### Create desired operation file
Start by reading the an existing operation file.

In [ ]:
op_manager = OperationManager(INPUT_DIR.joinpath('ContinuousCorn.operation'))
print(op_manager)

Remove the existing N fertilization and replace it with the one you want to apply, in our case, 80kg of NH4 and 20kg of NO3 applied on the 120th day of the year.

In [ ]:
# Operations are stored in a dictionary with (year, doy, operation_type) keys
# To remove some operations, we must specify a list of such keys
op_manager.delete_operations([(1, 110, 'FIXED_FERTILIZATION')])

# To specify the new operation we want to introduce, we must specify its (year, doy, operation_type) key
# and the corresponding value, which is another dictionary containing all the necessary parameters.
k = (1, 120, 'FIXED_FERTILIZATION')
v = {'SOURCE': 'UreaAmmoniumNitrate', 
     'MASS': 100.0, 
     'FORM': 'Liquid', 
     'METHOD': 'Broadcast', 
     'LAYER': 1.0, 
     'C_Organic': 0.0, 
     'C_Charcoal': 0.0, 
     'N_Organic': 0.0, 
     'N_Charcoal': 0.0, 
     'N_NH4': 0.8, 
     'N_NO3': 0.2, 
     'P_Organic': 0.0, 
     'P_CHARCOAL': 0.0, 
     'P_INORGANIC': 0.0, 
     'K': 0.0, 
     'S': 0.0}
op_manager.insert_new_operations({k: v})
print(op_manager)  # Notice the operations are already sorted by doy, which is important for cycles

# Save the new operation file
new_op_fname = INPUT_DIR.joinpath('ContinuousCornNew.operation')
op_manager.save(new_op_fname)

### Create desired control file
Subsequently, we must create a new control file to tell cycles to use our new operation file during the simulation. First we need to load the existing control file.


In [ ]:
ctrl_manager = ControlManager(INPUT_DIR.joinpath('ContinuousCorn.ctrl'))
print(ctrl_manager)

Subsequently, we change the OPERATION_FILE field (we also change the end year to have a one year simulation). Similarly to the operation manager, the control file manager stores the data in a dictionary.

In [ ]:
ctrl_manager.ctrl_dict['OPERATION_FILE'] = new_op_fname.name
ctrl_manager.ctrl_dict['SIMULATION_END_YEAR'] = 1980
print(ctrl_manager)

# Save
new_ctrl_fname = INPUT_DIR.joinpath('ContinuousCornNew.ctrl')
ctrl_manager.save(new_ctrl_fname)

### Run simulation and get results
Now we are ready to call cycles using our newly created control file as input. In the future, it may be nice to have a class to handle this?

In [ ]:
CYCLES_PATH.joinpath('Cycles').chmod(stat.S_IEXEC)  # Make cycles executable
subprocess.run(['./Cycles', '-b', new_ctrl_fname.stem], cwd=CYCLES_PATH, stdout=subprocess.DEVNULL)

Now we are ready to load the summary of the season that we just simulated. By default, Cycles stores them in a directory with the same name as the control file contained in CYCLES_PATH/output/. Note the season manager stores data in a pandas dataframe.

In [ ]:

sea_manager = SeasonManager(OUTPUT_DIR.joinpath(new_ctrl_fname.stem, 'season.dat'))
grain_yield = sea_manager.season_df.at[0, 'GRAIN YIELD']
print(grain_yield)

### Tidy up
Let's clean up after ourselves.

In [ ]:
# Remove newly created input files
new_op_fname.unlink(missing_ok=True)  
new_ctrl_fname.unlink(missing_ok=True)

# Remove output directory
shutil.rmtree(OUTPUT_DIR.joinpath('ContinuousCornNew'))

### Let's put this together!
Let's write a function that takes as input a list of tuples (doy, kg of NH4, kg of NO3) and outputs a the grain yield of the corresponding fertilization policy.

In [ ]:
def N2yield(fertilizations):
    
    # Operation
    print('Creating new operation file...')
    op_manager = OperationManager(INPUT_DIR.joinpath('ContinuousCorn.operation'))
    op_manager.delete_operations([(1, 110, 'FIXED_FERTILIZATION')]) # We should be checking if this deletes other fertilizations e.g. C and, in case, preserve those operations.
    
    new_operations = {}
    for fert in fertilizations:
        doy, NH4, NO3 = fert  # Should validate input
        mass = NH4 + NO3
        NH4_ratio = NH4 / mass
        NO3_ratio = NO3 / mass
        k = (1, doy, 'FIXED_FERTILIZATION')
        v = {'SOURCE': 'UreaAmmoniumNitrate', 
             'MASS': mass, 
             'FORM': 'Liquid', 
             'METHOD': 'Broadcast', 
             'LAYER': 1.0, 
             'C_Organic': 0.0, 
             'C_Charcoal': 0.0, 
             'N_Organic': 0.0, 
             'N_Charcoal': 0.0, 
             'N_NH4': NH4_ratio, 
             'N_NO3': NO3_ratio, 
             'P_Organic': 0.0, 
             'P_CHARCOAL': 0.0, 
             'P_INORGANIC': 0.0, 
             'K': 0.0, 
             'S': 0.0}
        new_operations.update({k:v})
    
    op_manager.insert_new_operations(new_operations)
    new_op_fname = INPUT_DIR.joinpath('ContinuousCornTemp.operation')

    op_manager.save(new_op_fname)
    
    # Control
    print('Creating new control file...')
    ctrl_manager = ControlManager(INPUT_DIR.joinpath('ContinuousCorn.ctrl'))
    ctrl_manager.ctrl_dict['OPERATION_FILE'] = new_op_fname.name
    ctrl_manager.ctrl_dict['SIMULATION_END_YEAR'] = 1980 # Not necessary
    new_ctrl_fname = INPUT_DIR.joinpath('ContinuousCornTemp.ctrl')
    ctrl_manager.save(new_ctrl_fname)
    
    # Cycles
    print('Running simulation...')
    subprocess.run(['./Cycles', '-b', new_ctrl_fname.stem], cwd=CYCLES_PATH, stdout=subprocess.DEVNULL)
    
    # Season
    print('Reading results...')
    sea_manager = SeasonManager(OUTPUT_DIR.joinpath(new_ctrl_fname.stem, 'season.dat'))
    grain_yield = sea_manager.season_df.at[0, 'GRAIN YIELD']
    
    # Clean up
    print('Tidying up...')
    # Remove newly created input files
    new_op_fname.unlink(missing_ok=True)  
    new_ctrl_fname.unlink(missing_ok=True)

    # Remove output directory
    shutil.rmtree(OUTPUT_DIR.joinpath(new_ctrl_fname.stem))
    
    return(grain_yield)
    
fertilizations = [(110, 150, 60), (150, 10, 20)]
N2yield(fertilizations)
    